In [13]:
library(sl3)
library(tmle3)
library(origami)
library(dplyr)

<h1>Definition of TMLE Helper Functions for Time Series </h1>
The probability density function of any given observation can be factorised into:

$$ p^N(o) = \prod_{t=1}^Np_{a(t)}(a(t)| \bar o(t-1))\prod_{t=1}^Np_{y(t)}(y(t)|\bar o(t-1), a(t))\prod_{t=1}^Np_{w(t)}(w(t)|\bar o(t-1), y(t), a(t)) \\
p^N(o) = g_{a(t)} \cdot q_{y(t)} \cdot q_{w(t)} $$ 

We assume $P_{O(t)|\bar O(t-1)}$ depends on $\bar O(t-1)$ throught the summary measure $C_o(t)$. We use the <em>conditional stationarity assumption</em>, i.e. the probability of an observation only depends on the summary measure and not $t$. 

$$
(c,o) \rightarrow p_{C_o(t)}(o | c) = \bar p(o | c) \text{ constant in } t
$$

We use the superlearning algorithm to find the initial fits $\theta_N = (\bar g, \bar q_y)$. As noted we can describe the density function conditioned on the summary measure.
$$
    p_{C_o(t)}(a,y,w)= \bar g(a | C_a(t)) \bar q_y(y| C_y(t))q_{w(t)}(w | C_w(t))
$$

Therefore for $q_{y(t)}$ the regression task is simply $(y(t)|C_y(t))$. Simple manipulation of the raw data can be put in such a way that we can the traditional superlearner pipeline by specifying $C_o(t) \in \bold W$. Example for $C_o(t)$ of dimension two is given below.

We manipulate the standard data table 
| W | A | Y |
|---|---|---|
| 1 | 0 | 1 |
| 0 | 1 | 1 |
| 1 | 1 | 0 |

to 

| W | A | Y | A_lag_1 | A_lag_2 | Y_lag_1 | Y_lag_2
|---|---|---| ---|---|---| --- |
| 1 | 0 | 1 | 0 | 0 | 0 | 0 |
| 0 | 1 | 1 | 0 | 0 | 1 | 0 |
| 1 | 1 | 0 | 1 | 0 | 1 | 1 |

Where for the start at the time series we input 0 as a placeholder if the lag goes results in a negative value for $t$. 

I see no reason as to why we can't stack every subject in the dataset as opposed to one single observed time series.

In [14]:
lag_column = function(column, lag){
    lag_col <- lapply(seq_len(lag), function(x) {
        Hmisc::Lag(column[, 1], x)
    })
    lag_cols <- data.frame(matrix(unlist(lag_col), nrow = length(lag_col[[1]])),
        stringsAsFactors = FALSE)
    return(lag_cols)
}


generate_lag_table = function(W, A, Y, lag){
    lag_As = lag_column(A, lag) 
    names(lag_As) <- paste0("A_lag_", seq(lag))

    lag_Ys = lag_column(Y, lag) 
    names(lag_Ys) <- paste0("Y_lag_", seq(lag))

    combined_lag_columns <- cbind.data.frame(lag_As, lag_Ys)
    combined_lag_columns[is.na(combined_lag_columns)] <- 0

    return(cbind.data.frame(W, A, Y, combined_lag_columns))
}


make_tmle_task = function(data, node_list, folds, lag=5){
    data_lag <- generate_lag_table(W = data.frame(W=data[, node_list$W]),
        A = data.frame(A=data[, node_list$A]),
        Y = data.frame(Y=data[, node_list$Y]),
        lag
    )

    # I think this fixes a bug in the original tstmle3, where data_lag has column Y, 
    # but node_list_lag refers to it as "cnt" in this example
    node_list_lag <- list(
        W = colnames(data_lag)[-(which(names(data_lag)=="A" | names(data_lag)=="Y"))],
        A = "A",
        Y = "Y"
    )

    ate_spec <- tmle_ATE(treatment_level = 1, control_level=0)

    return(ate_spec$make_tmle_task(data_lag, node_list_lag, folds=folds))
}

<h3>Define Learners</h3>

Paper 5.2 <em>"We advocate for online Super-learner based estimators based on the log-likelihood loss and appropriate CV for time-series (Rolling)"</em>.

<em>Personal Note: We're estimating the probability function of $y(t), g(t)$ so loglikelihood loss seems sensible</em>

In [15]:
# choose base learners
#lrnr_mean <- make_learner(Lrnr_mean)
lrnr_rf <- make_learner(Lrnr_ranger)
lrnr_mean <- make_learner(Lrnr_glm)
#lrnr_hal <- make_learner(Lrnr_hal9001)

# define metalearners appropriate to data types
ls_metalearner <- make_learner(Lrnr_nnls)
treatment_metalearner <- make_learner(
  Lrnr_solnp, 
  loss_function = loss_loglik_binomial,
  learner_function = metalearner_logistic_binomial
)

outcome_metalearner <- make_learner(
  Lrnr_solnp, 
  loss_function = loss_loglik_true_cat, # Different Loss function as Y is continous outcome not bounded to (0,1)?
  learner_function = metalearner_logistic_binomial
)

sl_A <- Lrnr_sl$new(
  learners = list(lrnr_mean, lrnr_rf),
  metalearner = treatment_metalearner 
)

sl_Y <- Lrnr_sl$new(
  learners = list(lrnr_mean, lrnr_rf),
  metalearner = outcome_metalearner 
)

learner_list <- list(A = sl_A, Y = sl_Y)

<h2>Run TMLE</h2>

In [16]:
# Sample Data Set
# data(bsds)
# data <- bsds
# node_list <- list(
#     W = "weathersit",
#     A = "workingday",
#     Y = "cnt"
# )

# Custom Data
data <- read.csv("data/experiments/experiment_one_DBCC14.csv")
#data$pm2_5 <- ifelse(data$pm2_5 >= 50, 1, 0)
node_list <- list(
    W = c("activity_type"),
    A = "pm2_5",
    Y = "breathing_rate"
)

head(data)

,breathing_rate,activity_type,pm2_5
,<dbl>,<int>,<dbl>
1,19.68266,0,3.095517
2,25.18890,0,3.304939
3,24.04258,0,2.941362
4,19.68266,0,2.716043
5,19.68266,1,3.184329
6,19.68266,1,3.957716


In [17]:
make_tmle_task = function(data, node_list, folds, lag=5){
    data_lag <- generate_lag_table(W = data.frame(W=data[, node_list$W]),
        A = data.frame(A=data[, node_list$A]),
        Y = data.frame(Y=data[, node_list$Y]),
        lag
    )

    data_lag$A <- ifelse(data_lag$A >= 50, 1, 0)

    A_cols <- colnames(data_lag)[grepl("A", colnames(data_lag))]
    # I think this fixes a bug in the original tstmle3, where data_lag has column Y, 
    # but node_list_lag refers to it as "cnt" in this example
    node_list_lag <- list(
        W = colnames(data_lag)[-(which(names(data_lag)=="A" | names(data_lag)=="Y"))],
        A = "A",
        Y = "Y"
    )

    ate_spec <- tmle_ATE(treatment_level = 1, control_level=0)
    return(ate_spec$make_tmle_task(data_lag, node_list_lag, folds=folds))
}

folds <- origami::make_folds(fold_fun = folds_rolling_origin, n=nrow(data), first_window = 100, validation_size = 30, gap = 0, batch = 30)
tmle_task <- make_tmle_task(data, node_list, folds, lag = 5)

In [19]:
att_spec <- tmle_ATT(treatment_level = 1, control_level=0)

In [20]:
ate_spec <- tmle_ATE(treatment_level = 1, control_level=0)

initial_likelihood <- ate_spec$make_initial_likelihood(
    tmle_task,
    learner_list
)

print(initial_likelihood)

W: Lf_emp
A: LF_fit
Y: LF_fit


In [24]:
initial_likelihood$factor_list[["Y"]]

Y: LF_fit

In [21]:
updater <- ate_spec$make_updater(cvtmle=TRUE)
targeted_likelihood <- ate_spec$make_targeted_likelihood(initial_likelihood, updater)
tmle_params <- ate_spec$make_params(tmle_task, targeted_likelihood)

tmle_params

[[1]]
Param_ATE: ATE[Y_{A=1}-Y_{A=0}]


In [22]:
tmle_fit <- fit_tmle3(tmle_task, targeted_likelihood, tmle_params, updater)
#One-Step
EIC <- tmle_fit$estimates[[1]]$IC
oneStepEst <- (tmle_fit$initial_psi + mean(EIC))

oneStepEst

[1] -0.03905373

In [39]:
class(tmle_fit)

[1] "tmle3_Fit" "R6"

Currently the TMLE is estimate the causal effect of a single time-point intervention $A(t)$ on subsequent outcome $Y(t)$, conditional on history. 
$$ \Psi_{C_o(t)}(\bar q_y) = E(Y(t) | C_o(t), A(t) = 1) = \int y\bar q_y(y|C_o(t),1)d \mu_y(y)$$
$$ E(Y(t) | C_o(t), A(t) = 1) - E(Y(t) | C_o(t), A(t) = 0)$$

In [28]:
tmle_fit

A tmle3_Fit that took 1 step(s)
   type                param     init_est  tmle_est        se     lower
1:  ATE ATE[Y_{A=1}-Y_{A=0}] -0.004562555 0.2001411 0.2412055 -0.272613
       upper psi_transformed lower_transformed upper_transformed
1: 0.6728952       0.2001411         -0.272613         0.6728952

In [29]:
source("tstmle.r")
ate_spec <- tmle_ATE(treatment_level = 1, control_level=0)

run_tstmle(ate_spec, data, node_list, markov_order=30)

In [ ]:
0.65 - 0.25

0.25 - 0.4

[1] 0.4

[1] -0.15

<h2>Prediction Counterfactual Outcome - Currently usign Super Learner Regression :( </h2>

From Targeted Learning in Data Science: 19.7.1 
$$ \psi(P) = E_pE_p(Y | A = 1, W) $$
Loosely, the Expectation of treatment average over all covariates.

Want I want is to plot a time series of observed data and the counterfactual of there being treatment. For that I would want to use 
$$ \bar Q(W) = E_P(Y| A=1, W) $$
where I specify the $W$ to generate the counterfactual outcome.

NB:
$$\psi(P) = \bar Q_w\bar Q = \int \bar Q(w) d \bar Q_w(w)$$

So far: I can only figure out how to do this use the superlearner :(

In [2]:
get_sl3_task = function(data, node_list, folds, lag=5){
    data_lag <- generate_lag_table(W = data.frame(W=data[, node_list$W]),
        A = data.frame(A=data[, node_list$A]),
        Y = data.frame(Y=data[, node_list$Y]),
        lag
    )

    # I think this fixes a bug in the original tstmle3, where data_lag has column Y, 
    # but node_list_lag refers to it as "cnt" in this example
    node_list_lag <- list(
        W = colnames(data_lag)[-(which(names(data_lag)=="A" | names(data_lag)=="Y"))],
        A = "A",
        Y = "Y"
    )


    return(make_sl3_Task(data = data_lag, outcome ="Y", covariates = node_list_lag$W, folds=folds))
}

In [9]:
folds <- origami::make_folds(fold_fun = folds_rolling_origin, n=nrow(data), first_window = 100, validation_size = 30, gap = 0, batch = 30)

sl_task <- get_sl3_task(data, node_list, folds, lag = 5)

#lrn_hal <- Lrnr_hal9001$new(max_degree = 2, num_knots = c(3,2))
lrn_glm <- Lrnr_glm$new()
lrn_mean <- Lrnr_mean$new()

# Why does lrn_hal increase Run Time by 3 orders of magnitude???

stack = Stack$new(lrn_mean, lrn_glm)
sl <- Lrnr_sl$new(learners = stack, metalearner = Lrnr_nnls$new())

sl_fit <- sl$train(sl_task)

In [10]:
counterfactual_data <- read.csv("data/experiments/experiment_one_counterfactual_DBCC17.csv")
counterfactual_data$pm2_5 <- ifelse(counterfactual_data$pm2_5 >= 0, 1, 0) # Counterfactual: Treatment for sustained period
head(counterfactual_data)

,breathing_rate,activity_type,pm2_5
,<dbl>,<int>,<dbl>
1,31.71102,0,1
2,27.78943,0,1
3,20.90418,0,1
4,18.99838,0,1
5,NA,1,1
6,NA,1,1


In [11]:
counterfactual_task <- get_sl3_task(counterfactual_data, node_list, folds, lag = 5)
counterfactual_pred <- sl_fit$predict(counterfactual_task)

Warning message in process_data(data, nodes, column_names = column_names, flag = flag, :
"Missing outcome data detected. This is okay for prediction, but will likely break training. 
 You can drop observations with missing outcomes by setting drop_missing_outcome=TRUE in make_sl3_Task."


In [12]:
counterfactual_pred

[1] 12.94572 16.80724 18.67353 17.84736 18.24488 16.95857 15.09278 14.61383
  [9] 16.62041 18.51814 15.44807 14.11922 15.17887 14.26292 12.94583 16.27780
 [17] 18.16234 15.19201 16.91098 19.52148 18.06808 17.24030 17.67413 17.61192
 [25] 17.99607 18.81729 18.79344 18.69689 19.27674 19.62864 19.56943 19.30762
 [33] 18.90136 18.40548 18.69205 18.59990 18.32801 18.91693 19.05470 19.23119
 [41] 19.12147 19.01866 19.45156 19.87667 18.67710 16.56248 15.13787 14.84294
 [49] 13.81307 12.86698 12.23615 12.23615 12.94583 12.94583 12.94583 12.94583
 [57] 12.23615 12.94583 15.55454 17.03980 16.62133 17.78949 18.99850 18.83506
 [65] 18.67075 18.17785 18.73614 18.71198 18.69868 18.73327 19.22339 19.05377
 [73] 19.28544 19.37300 19.28932 18.82275 18.96278 19.09392 18.78951 19.09834
 [81] 19.03904 19.47565 19.84239 19.78100 19.63037 19.71447 19.73833 19.56174
 [89] 19.21509 19.33956 19.17134 18.95725 16.45900 17.46644 19.65444 18.50462
 [97] 18.05308 19.07672 19.16485 16.38377 14.83798 14.62836 13.87646 12.94583
[105] 12.94583 15.81730 15.11785 13.14659 14.03088 14.15554 12.86698 12.86698
[113] 12.86698 12.86698 12.86698 12.86698 12.86698 12.86698 12.86698 12.86698
[121] 12.86698

In [ ]:
#write.csv(counterfactual_pred, file="data/experiments/experiment_one_counterfactual_outcome.csv", row.names=FALSE)